In [1]:
from pathlib import Path
from pytz import timezone
import datetime
import time

In [2]:
from visualization import BokehCandleStickDB, static_candlestick
from get_stock_price import StockDatabase

In [3]:
import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

In [4]:
db_path = Path("db/stock_db") / Path("stock.db")
stock_db = StockDatabase(db_path)

In [5]:
stock_name = ["4755"] # 楽天
stock_db.stock_in(stock_name)

True

In [6]:
stock_timestamp_df = stock_db.stock_timestamp(stock_name)
stock_timestamp_df

,min_datetime,max_datetime,column_name
0,2020-11-02 00:00:00+00:00,2020-11-13 05:59:00+00:00,4755


In [7]:
day_before = stock_timestamp_df.loc[0,"min_datetime"] + datetime.timedelta(days=10)

# 日時の取得
jst_timezone = timezone("Asia/Tokyo")
start_time = jst_timezone.localize(datetime.datetime(day_before.year, day_before.month, day_before.day, 9, 0, 0))
#start_time = jst_timezone.localize(datetime.datetime(day_before.year, day_before.month, day_before.day, 12, 30, 0))
end_time = jst_timezone.localize(datetime.datetime(day_before.year, day_before.month, day_before.day, 12, 30, 0))
#end_time = jst_timezone.localize(datetime.datetime(day_before.year, day_before.month, day_before.day, 15, 0, 0))

### 静的なグラフの場合 

In [8]:
span_df = stock_db.search_span(stock_name, start_time, end_time, freq_str="5T",to_tokyo=True)
ohlc_dict = {"Open":"Open_4755", "High":"High_4755", "Low":"Low_4755", "Close":"Close_4755"}

p = static_candlestick(span_df, ohlc_dict)
bokeh.io.show(p)

### 動的なグラフの場合 

In [9]:
bokeh_candle_stick = BokehCandleStickDB(stock_db, 
                                        stock_name, 
                                        ohlc_dict, 
                                        initial_start_datetime=start_time,
                                        initial_end_datetime=end_time,
                                        freq_str="5T",
                                        y_axis_margin=10,
                                        to_tokyo=True,
                                        data_left_times=5,
                                        use_formatter=True,
                                       )

In [10]:
t = bokeh.io.show(bokeh_candle_stick.dp, notebook_handle=True)
bokeh_candle_stick.set_t(t)
for i in range(1000):
    time.sleep(0.5)
    bokeh_candle_stick.update()

DatabaseError: Execution failed on sql '
        select timestamp, Close_4755, High_4755, Low_4755, Open_4755, Volume_4755 from table_1 where timestamp >= ? and timestamp < ?
        ': Error binding parameter 0 - probably unsupported type.